In [28]:
import os
import pandas as pd
import dtale
import sys, os
project_root = os.path.abspath("..")  # adjust based on your directory structure
if project_root not in sys.path:
    sys.path.append(project_root)
    
from FeatureCleaning.CleanDSDTale import export_clean_df


def main():
    
    # 1. Load the cleaned issue-level DataFrame
    df = export_clean_df()
    
    # 2. Compute time-to-resolution in hours
    df['time_to_resolution'] = (
        pd.to_datetime(df['fields.resolutiondate']) - 
        pd.to_datetime(df['fields.created'])
    ).dt.total_seconds() / 3600  # in hours

    item_counts_per_project = df.groupby("fields.project.id").size().reset_index(name="item_count")

    # 3. Aggregate overall project-level features (transformations and mean/median values)
    agg_df = df.groupby(['fields.project.id', 'fields.project.name']).agg({
        'time_to_resolution': ['mean', 'median', 'min', 'max', 'sum'],
        'changelog_count_status': ['sum', 'mean'],
        'changelog_count_assignee': ['sum', 'mean'],
        'comment_count': ['sum', 'mean'],
    }).reset_index()
    
    # Flatten the MultiIndex columns
    agg_df.columns = [
        '_'.join(col).strip() if col[1] else col[0]
        for col in agg_df.columns.values
    ]

    df = pd.merge(agg_df, item_counts_per_project, on="fields.project.id", how="left")
    
    # 6. Optionally display the final DataFrame using D-Tale for interactive exploration
    print(df.head())
    d = dtale.show(df, ignore_duplicate=True, allow_cell_edits=False)
    d.open_browser()

if __name__ == "__main__":
    main()


Processing repository: Hyperledger ...
Processing repository: SecondLife ...



/Users/diegodias/Documents/Projects/JiraDataset/FeatureCleaning/CleanDSDTale.py:394: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/diegodias/Documents/Projects/JiraDataset/FeatureCleaning/CleanDSDTale.py:394: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Data processed. Launching D-Tale session...


/opt/anaconda3/envs/my_mongo_env/lib/python3.9/site-packages/dtale/utils.py:911: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2011-01-27 09:28:42', '2013-09-04 20:00:43', '2014-12-01 19:49:44',
 '2013-09-04 14:16:28', '2011-09-19 14:24:17', '2011-03-18 15:34:24',
 '2014-03-06 20:08:14', '2011-04-26 12:41:56', '2010-09-17 19:57:06',
 '2010-10-07 13:27:41',
 ...
 '2019-10-08 11:47:56', '2018-09-11 16:22:44', '2021-03-15 19:37:10',
 '2019-07-30 02:42:05', '2020-01-16 13:32:12', '2019-08-31 14:22:12',
 '2019-10-08 13:53:43', '2018-05-31 01:36:23', '2019-03-08 16:33:04',
 '2017-03-14 20:30:32']
Length: 258, dtype: datetime64[ns]' has dtype incompatible with datetime64[ns, UTC], please explicitly cast to a compatible dtype first.

/opt/anaconda3/envs/my_mongo_env/lib/python3.9/site-packages/dtale/utils.py:911: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a futu

✅ D-Tale session launched successfully.
   fields.project.id  fields.project.name  time_to_resolution_mean  \
0            10001.0             Sawtooth              6275.539630   
1            10002.0               Fabric              2270.089098   
2            10100.0  Blockchain Explorer              2157.222698   
3            10200.0                Cello              1469.208924   
4            10244.0            Snowstorm              4962.281944   

   time_to_resolution_median  time_to_resolution_min  time_to_resolution_max  \
0                 880.896250              339.057222            32903.286111   
1                 409.525833                0.033333            25287.602222   
2                 354.641944              139.496667            12161.649722   
3                 166.355972               27.941944             8027.217778   
4                2781.890000              312.760278            19391.665278   

   time_to_resolution_sum  changelog_count_status_sum  \
0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/var/folders/lt/2k0m0j2d6xqg0wz_673wgqt00000gn/T/ipykernel_3225/1749906111.py:18: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 

2025-02-25 07:44:21,282 - INFO     - Executing shutdown...
2025-02-25 07:44:21,283 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
/opt/anaconda3/envs/my_mongo_env/lib/python3.9/site-packages/dtale/app.py:445: FutureWarning:

`torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead

